In [4]:
'''
ChenNet
fast convolutional model with:
    unrolling convolutional layer
    sketch pooling layer
'''


# data loading and preprocessing of MNIST
# directly download MNIST from website
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from struct import unpack
import time

def read_image(path):
    with open(path, 'rb') as f:
        magic, num, rows, cols = unpack('>4I',f.read(16))
        img = np.fromfile(f, dtype=np.uint8).reshape(num, rows, cols, 1)   # regulate format and channels
    return img

def read_label(path):
    with open(path, 'rb') as f:
        magic, num = unpack('>2I',f.read(8))
        label = np.fromfile(f, dtype=np.uint8)
    return label

def normalize_image(image):
    img = image.astype(np.float32)/255.0
    return img

def one_hot_label(label):
    lab = np.zeros((label.size, 10))
    for i, row in enumerate(lab):
        row[label[i]] = 1
    return lab

# load data and preprocessing 

def dataset_loader():
    train_image = read_image(r'C:\Users\41713\CIS\Python_LeNet_UnderlyingImplementation-master\Python_LeNet_UnderlyingImplementation-master\卷积神经网络LeNet实现\mnist\train-images.idx3-ubyte')
    train_label = read_label(r'C:\Users\41713\CIS\Python_LeNet_UnderlyingImplementation-master\Python_LeNet_UnderlyingImplementation-master\卷积神经网络LeNet实现\mnist\train-labels.idx1-ubyte')
    test_image = read_image(r'C:\Users\41713\CIS\Python_LeNet_UnderlyingImplementation-master\Python_LeNet_UnderlyingImplementation-master\卷积神经网络LeNet实现\mnist\t10k-images.idx3-ubyte')
    test_label = read_label(r'C:\Users\41713\CIS\Python_LeNet_UnderlyingImplementation-master\Python_LeNet_UnderlyingImplementation-master\卷积神经网络LeNet实现\mnist\t10k-labels.idx1-ubyte')

    train_image = normalize_image(train_image)
    train_label = one_hot_label(train_label)
    train_label = train_label.reshape(train_label.shape[0], train_label.shape[1], 1)

    test_image = normalize_image(test_image)
    test_label = one_hot_label(test_label)
    test_label = test_label.reshape(test_label.shape[0], test_label.shape[1], 1)
    
    return train_image, train_label, test_image, test_label

# image: num×rows×cols×1，ranging from 0-1
# label: num×class_num×1
train_image, train_label, test_image, test_label = dataset_loader()


# padding。input from 28*28 to 32*32
def padding(image, zero_num):
    if len(image.shape) == 4:
        image_padding = np.zeros((image.shape[0],image.shape[1]+2*zero_num,image.shape[2]+2*zero_num,image.shape[3]))
        image_padding[:,zero_num:image.shape[1]+zero_num,zero_num:image.shape[2]+zero_num,:] = image
    elif len(image.shape) == 3:
        image_padding = np.zeros((image.shape[0]+2*zero_num, image.shape[1]+2*zero_num, image.shape[2]))
        image_padding[zero_num:image.shape[0]+zero_num, zero_num:image.shape[1]+zero_num,:] = image
    else:
        print("dimensional mismatch!")
        sys.exit()
    return image_padding

train_image = padding(train_image,2)# padding to conform to the input of 60000*32*32*1
test_image = padding(test_image,2)

In [5]:
print(train_image.shape)
print(train_label.shape)

(60000, 32, 32, 1)
(60000, 10, 1)


In [6]:
def conv(img, conv_filter):
   
    if len(img.shape)!=3 or len(conv_filter.shape)!=4:
        print("dimensional mismatch!")
        sys.exit()
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("channel mismatch!")
        sys.exit()
        
    img_h, img_w, img_ch = img.shape
    filter_num, filter_h, filter_w, img_ch = conv_filter.shape
    feature_h = img_h - filter_h + 1
    feature_w = img_w - filter_w + 1

    # feature map initialization
    img_out = np.zeros((feature_h, feature_w, filter_num))
    img_matrix = np.zeros((feature_h*feature_w, filter_h*filter_w*img_ch))
    filter_matrix = np.zeros((filter_h*filter_w*img_ch, filter_num))
    
    # input tensor to matrix
    for i in range(feature_h*feature_w):
        for j in range(img_ch):
            img_matrix[i, j*filter_h*filter_w:(j+1)*filter_h*filter_w] = \
            img[np.uint16(i/feature_w):np.uint16(i/feature_w+filter_h),np.uint16(i%feature_w):np.uint16(i%feature_w+filter_w),j].reshape(filter_h*filter_w)
    
    # kernel tensor to matrix
    for i in range(filter_num):
        filter_matrix[:,i] = conv_filter[i,:].reshape(filter_w*filter_h*img_ch)
    
    feature_matrix = np.dot(img_matrix, filter_matrix)
    
    # output matrix to tensor
    for i in range(filter_num):
        img_out[:,:,i] = feature_matrix[:,i].reshape(feature_w, feature_h)
    
    return img_out


def conv_(img, conv_filter):
    
    # 2D image valid convolution
    img_h, img_w = img.shape
    filter_h, filter_w = conv_filter.shape
    feature_h = img_h - filter_h + 1
    feature_w = img_w - filter_w + 1
    
    img_matrix = np.zeros((feature_h*feature_w, filter_h*filter_w))
    for i in range(feature_h*feature_w):
        img_matrix[i:] = \
        img[np.uint16(i/feature_w):np.uint16(i/feature_w+filter_h),np.uint16(i%feature_w):np.uint16(i%feature_w+filter_w)].reshape(filter_w*filter_h)
    filter_matrix = conv_filter.reshape(filter_h*filter_w,1)
    
    img_out = np.dot(img_matrix, filter_matrix)
    
    img_out = img_out.reshape(feature_h,feature_w)
            
    return img_out


def conv_cal_w(out_img_delta, in_img):
    # gradient calculation of convolutional layer
    nabla_conv = np.zeros([out_img_delta.shape[-1], in_img.shape[0]-out_img_delta.shape[0]+1, 
                           in_img.shape[1]-out_img_delta.shape[1]+1, in_img.shape[-1]])
    for filter_num in range(nabla_conv.shape[0]):
        for ch_num in range(nabla_conv.shape[-1]):
            nabla_conv[filter_num,:,:,ch_num] = conv_(in_img[:,:,ch_num], out_img_delta[:,:,filter_num])
    return nabla_conv

def conv_cal_b(out_img_delta):
    nabla_b = np.zeros((out_img_delta.shape[-1],1))
    for i in range(out_img_delta.shape[-1]):
        nabla_b[i] = np.sum(out_img_delta[:,:,i])
    return nabla_b


def relu(feature):
    '''Relu activation function, using when:
    convolutional layer，feature is a 3D tensor，，[row, col, channel]
    fully-connected layer，feature is a column-wise vector'''
    relu_out = np.zeros(feature.shape)
    
    if len(feature.shape) > 2:
        for ch_num in range(feature.shape[-1]):
            for r in range(feature.shape[0]):
                for c in range(feature.shape[1]):
                    relu_out[r,c,ch_num] = max(feature[r,c,ch_num], 0)
    else:
        for r in range(feature.shape[0]):
            relu_out[r,0] = max(feature[r,0], 0)
    return relu_out


def relu_prime(feature):  # relu prime
    '''relu prime，derivative of interval is considered 0'''
    
    relu_prime_out = np.zeros(feature.shape)
    
    if len(feature.shape) > 2:
        for ch_num in range(feature.shape[-1]):
            for r in range(feature.shape[0]):
                for c in range(feature.shape[1]):
                    if feature[r,c,ch_num] > 0:
                        relu_prime_out[r,c,ch_num] = 1

    else:
        for r in range(feature.shape[0]):
            if feature[r,0] > 0:
                relu_prime_out[r, 0] = 1
            
    return relu_prime_out


def pool(feature, size=2, stride=2):
    '''sketch pooling
    keep the sketching matrix at the same time for back propagation'''
    pool_out = np.zeros([np.uint16((feature.shape[0]-size)/stride+1),
                        np.uint16((feature.shape[1]-size)/stride+1),
                         feature.shape[2]])
    sketch_mtx = np.zeros([np.uint16((feature.shape[0]-size)/stride+1),
                        feature.shape[1],
                         feature.shape[2]])
    for i in range(feature.shape[-1]):        
        pool_out[:,:,i], sketch_mtx[:,:,i] = countSketch(feature[:,:,i], pool_out.shape[0])
    return pool_out, sketch_mtx

def countSketch(input, k):
    [r, c] = input.shape
    seed = np.random.randint(k, size = c)
    U = np.zeros((k,c))
    
    for i in range(c):
        if np.random.random()>=0.50:
            U[seed[i],i] = 1
        else:
            U[seed[i],i] = -1
    
    sketch1 = np.dot(U, input)
    sketch2 = np.dot(sketch1, np.transpose(U))
    #sketch2 = np.dot(sp.csr_matrix(sketch1), sp.csr_matrix(np.transpose(U))).todense()
    
    return sketch2, U

def gaussianSketch(input, k):
    [r, c] = input.shape
    U = np.random.randn(k, c)
    sketch1 = np.dot(U, input)
    sketch2 = np.dot(sketch1, np.transpose(U))
    return sketch2, U

def pool_delta_error_bp(pool_out_delta, sketch_mtx, size=2, stride=2):
    
    delta = np.zeros([np.uint16((pool_out_delta.shape[0]-1)*stride+size),
                      np.uint16((pool_out_delta.shape[1]-1)*stride+size),
                      pool_out_delta.shape[2]])
    d = delta
    for i in range(delta.shape[-1]):
        #d[:,:,i] = sp.csr_matrix(np.transpose(sketch_mtx[:,:,i]))*(sp.csr_matrix(pool_out_delta[:,:,i])*sp.csr_matrix(sketch_mtx[:,:,i])).todense()
        d[:,:,i] = np.dot(np.transpose(sketch_mtx[:,:,i]), np.dot(np.transpose(pool_out_delta[:,:,i]), sketch_mtx[:,:,i]))
    delta = d
    return delta

def rot180(conv_filters):
    rot180_filters = np.zeros((conv_filters.shape))
    for filter_num in range(conv_filters.shape[0]):
        for img_ch in range(conv_filters.shape[-1]):
            rot180_filters[filter_num,:,:,img_ch] = np.flipud(np.fliplr(conv_filters[filter_num,:,:,img_ch]))
    return rot180_filters
                
    
def soft_max(z):
        
    tmp = np.max(z)
    z -= tmp  # element zooming, avoid overfolw
    z = np.exp(z)
    tmp = np.sum(z)
    z /= tmp
    
    return z

def add_bias(conv, bias):
    if conv.shape[-1] != bias.shape[0]:
        print("error when adding bias")
    else:
        for i in range(bias.shape[0]):
            conv[:,:,i] += bias[i,0]
    return conv


In [7]:
class ConvNet(object):
    
    def __init__(self):
        
        '''
        2 conv layers，2 pooling layers，3 fully-connected layers'''
        self.filters = [np.random.randn(6, 5, 5, 1)] #in:32*32*1 out:28*28*6 pooling out:14*14*6
        self.filters_biases = [np.random.randn(6,1)]
        self.filters.append(np.random.randn(16, 5, 5, 6)) #in:14*14*6 out:10*10*16 pooling out:5*5*16
        self.filters_biases.append(np.random.randn(16,1))
        
        self.weights = [np.random.randn(120,400)]
        self.weights.append(np.random.randn(84,120))
        self.weights.append(np.random.randn(10,84))
        self.biases = [np.random.randn(120,1)]
        self.biases.append(np.random.randn(84,1))
        self.biases.append(np.random.randn(10,1))
        
    
    
    def feed_forward(self, x):
        #first conv
        conv1 = add_bias( conv(x, self.filters[0]), self.filters_biases[0] )
        relu1 = relu(conv1)
        pool1, pool1_max_locate = pool(relu1)
        
        #second conv
        conv2 = add_bias( conv(pool1, self.filters[1]), self.filters_biases[1])
        relu2 = relu(conv2)
        pool2, pool2_max_locate = pool(relu2)
        
        #reshape to pull straight
        straight_input = pool2.reshape(pool2.shape[0] * pool2.shape[1] * pool2.shape[2], 1)
        
        #first fully-connected layer
        full_connect1_z = np.dot(self.weights[0], straight_input) + self.biases[0]
        full_connect1_a = relu(full_connect1_z)
        
        #second fc layer
        full_connect2_z = np.dot(self.weights[1], full_connect1_a) + self.biases[1]
        full_connect2_a = relu(full_connect2_z)
        
        #third fc layer (output layer)
        full_connect3_z = np.dot(self.weights[2], full_connect2_a) + self.biases[2]
        full_connect3_a = soft_max(full_connect3_z)
        return full_connect3_a
    
    def evaluate(self, images, labels):
        result = 0
        for img, lab in zip(images, labels):
            predict_label = self.feed_forward(img)
            if np.argmax(predict_label) == np.argmax(lab):
                result += 1
        return result
    
    def SGD(self, train_image, train_label, epochs, mini_batch_size, eta):
        '''Stochastic gradiend descent'''
        batch_num = 0
        for j in range(epochs):
           
            mini_batches_image = [train_image[k:k+mini_batch_size] for k in range(0, len(train_image), mini_batch_size)]
            mini_batches_label = [train_label[k:k+mini_batch_size] for k in range(0, len(train_label), mini_batch_size)]
            start = time.time()
            for mini_batch_image, mini_batch_label in zip(mini_batches_image, mini_batches_label):
                batch_num += 1
                if batch_num * mini_batch_size > len(train_image):
                    batch_num = 1
                
                self.update_mini_batch(mini_batch_image, mini_batch_label, eta, mini_batch_size)
                
                if batch_num % 10 == 0:
                    print("after {0} training batch: accuracy is {1}/{2}".format(batch_num, self.evaluate(train_image[0:100], train_label[0:100]), len(train_image[0:100])))                    
                    interval = time.time()-start
                    print("time:" + str(interval))
                
                print("\rEpoch{0}:{1}/{2}".format(j+1, batch_num*mini_batch_size, len(train_image)), end=' ')
            
            print("After epoch{0}: accuracy is {1}/{2}".format(j+1, self.evaluate(test_image, test_label), len(test_image)))
                
    def update_mini_batch(self, mini_batch_image, mini_batch_label, eta, mini_batch_size):
        '''batch updating'''
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        nabla_f = [np.zeros(f.shape) for f in self.filters]
        nabla_fb = [np.zeros(fb.shape) for fb in self.filters_biases]
        
        for x,y in zip(mini_batch_image, mini_batch_label):
            delta_nabla_w, delta_nabla_b, delta_nabla_f, delta_nabla_fb = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_f = [nf+dnf for nf, dnf in zip(nabla_f, delta_nabla_f)]
            nabla_fb = [nfb + dnfb for nfb, dnfb in zip(nabla_fb, delta_nabla_fb)]
        self.weights = [w-(eta/mini_batch_size)*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/mini_batch_size)*nb for b, nb in zip(self.biases, nabla_b)]
        self.filters = [f-(eta/mini_batch_size)*nf for f, nf in zip(self.filters, nabla_f)]
        self.filters_biases = [fb-(eta/mini_batch_size)*nfb for fb, nfb in zip(self.filters_biases, nabla_fb)]
    
    def backprop(self, x, y):
        
        '''single input back propagation process'''
        
        #forward, deriving interval parameters
        #conv1
        conv1 = add_bias( conv(x, self.filters[0]), self.filters_biases[0] )
        relu1 = relu(conv1)
        pool1, pool1_max_locate = pool(relu1)

        #conv2
        conv2 = add_bias( conv(pool1, self.filters[1]), self.filters_biases[1] )
        relu2 = relu(conv2)
        pool2, pool2_max_locate = pool(relu2)
        
        #pull straight
        straight_input = pool2.reshape(pool2.shape[0] * pool2.shape[1] * pool2.shape[2], 1)
        
        #fc1
        full_connect1_z = np.dot(self.weights[0], straight_input) + self.biases[0]
        full_connect1_a = relu(full_connect1_z)
        
        #fc2
        full_connect2_z = np.dot(self.weights[1], full_connect1_a) + self.biases[1]
        full_connect2_a = relu(full_connect2_z)
        
        #fc3 (out)
        full_connect3_z = np.dot(self.weights[2], full_connect2_a) + self.biases[2]
        full_connect3_a = soft_max(full_connect3_z)
            
        # cross entropy, using softmax as activation function, delta is the euclidien distace between prediction and label
        delta_fc3 = full_connect3_a - y
        delta_fc2 = np.dot(self.weights[2].transpose(), delta_fc3) * relu_prime(full_connect2_z)
        delta_fc1 = np.dot(self.weights[1].transpose(), delta_fc2) * relu_prime(full_connect1_z)
        delta_straight_input = np.dot(self.weights[0].transpose(), delta_fc1)# 这里没有激活函数？
        delta_pool2 = delta_straight_input.reshape(pool2.shape)
        
        delta_conv2 = pool_delta_error_bp(delta_pool2, pool2_max_locate) * relu_prime(conv2)
        
        delta_pool1 = conv(padding(delta_conv2, self.filters[1].shape[1]-1), rot180(self.filters[1]).swapaxes(0,3))
        
        delta_conv1 = pool_delta_error_bp(delta_pool1, pool1_max_locate) * relu_prime(conv1)
        
        
        
        #derivitives
        nabla_w2 = np.dot(delta_fc3, full_connect2_a.transpose())
        nabla_b2 = delta_fc3
        nabla_w1 = np.dot(delta_fc2, full_connect1_a.transpose())
        nabla_b1 = delta_fc2
        nabla_w0 = np.dot(delta_fc1, straight_input.transpose())
        nabla_b0 = delta_fc1
        
        
        nabla_filters1 = conv_cal_w(delta_conv2, pool1) 
        nabla_filters0 = conv_cal_w(delta_conv1, x)
        nabla_filters_biases1 = conv_cal_b(delta_conv2)
        nabla_filters_biases0 = conv_cal_b(delta_conv1)
        
        nabla_w = [nabla_w0, nabla_w1, nabla_w2]
        nabla_b = [nabla_b0, nabla_b1, nabla_b2]
        nabla_f = [nabla_filters0, nabla_filters1]
        nabla_fb = [nabla_filters_biases0, nabla_filters_biases1]
        return nabla_w, nabla_b, nabla_f, nabla_fb

In [ ]:
net = ConvNet()
net.SGD(train_image, train_label, 5, 10, 3e-5) 

Epoch1:90/60000 after 10 training batch: accuracy is 7/100
time:6.363810777664185
Epoch1:190/60000 after 20 training batch: accuracy is 7/100
time:12.814523696899414
Epoch1:290/60000 after 30 training batch: accuracy is 12/100
time:19.190070152282715
Epoch1:390/60000 after 40 training batch: accuracy is 17/100
time:25.744056701660156
Epoch1:490/60000 after 50 training batch: accuracy is 7/100
time:32.09346961975098
Epoch1:590/60000 after 60 training batch: accuracy is 7/100
time:38.47032141685486
Epoch1:690/60000 after 70 training batch: accuracy is 7/100
time:44.94784212112427
Epoch1:790/60000 after 80 training batch: accuracy is 10/100
time:51.324997663497925
Epoch1:890/60000 after 90 training batch: accuracy is 9/100
time:58.03600215911865
Epoch1:990/60000 after 100 training batch: accuracy is 7/100
time:64.35456657409668
Epoch1:1030/60000 